# Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

#### Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

#### Важно:

   * Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
   * Если товар не встречался в обучающей выборке, его популярность равна 0.
   * Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
   * Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

#### Задание

   * На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
   * Реализуйте два алгоритма рекомендаций:

   * сортировка просмотренных id по популярности (частота появления в просмотренных),
   * сортировка просмотренных id по покупаемости (частота появления в покупках).

3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

Дополнительные вопросы

   * Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.
   * Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.
   
#### Данные
   https://d3c33hcgiwev3.cloudfront.net/_b82f058e50d7830fda44c75942e2fb84_coursera_sessions_train.txt?Expires=1500854400&Signature=HqWv6iKlvKB1XAd2UBKMq7b-WnMo~6KAmuPgFpo2CGMO89s1zEVE5CPxHUIi1EnxpgCAE-czteNSEeW7UeJrnwR4617ot-V-NSUsZUhAZqtY1cmv-V1xyPwZcowCJgiIqfrYPn6~c41ByvE4bdGvy~U65p8Q6nd0stAetoXalrk_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A
   
   https://d3c33hcgiwev3.cloudfront.net/_b82f058e50d7830fda44c75942e2fb84_coursera_sessions_test.txt?Expires=1500854400&Signature=i1rhMx8z8XIpGBAS-QtV~8uTTvA62pj5uqvWpgYl8NqJJWQhb1AcoCsSl6pVTTB~sViSSwG7fWRY6qo2NnuHsgACrj~VY7W4FFxC9Gcev4xwjIC5HqOR8SHDF0-zZtvmeKmaECYNOhPrBYZewcNgyo5B9N3oSYzHw0bgUjUI8AE_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A

## Сортировка по популярности

### Тренировочная выборка

In [228]:
import pandas as pd

In [262]:
train = pd.read_csv("coursera_sessions_train.txt", sep=';', header = 0, names = ["view", "buy"])

In [263]:
train.head()

,view,buy
0,"9,10,11,9,11,12,9,11",NaN
1,"16,17,18,19,20,21",NaN
2,"24,25,26,27,24",NaN
3,"34,35,36,34,37,35,36,37,38,39,38,39",NaN
4,42,NaN


In [264]:
def to_list(string):
    return[int(x) for x in string.split(',')]

In [265]:
train.view = map(to_list, train.view)

In [266]:
train.head()

,view,buy
0,"[9, 10, 11, 9, 11, 12, 9, 11]",NaN
1,"[16, 17, 18, 19, 20, 21]",NaN
2,"[24, 25, 26, 27, 24]",NaN
3,"[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]",NaN
4,[42],NaN


Подсчет частот всех просмотренных товаров

In [267]:
freq_view_all = dict()

for session_views in train.view:
    for i in range(len(session_views)):
        if session_views[i] not in freq_view_all:
            freq_view_all[session_views[i]] = 1
        else:
            freq_view_all[session_views[i]] += 1

In [268]:
train.dropna(inplace = True)

In [269]:
train.buy = map(to_list, train.buy)

In [270]:
train.head()

,view,buy
6,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]"
9,"[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]",[86]
18,"[138, 198, 199, 127]",[199]
29,"[303, 304, 305, 306, 307, 308, 309, 310, 311, ...",[303]
32,"[352, 353, 352]",[352]


Список купленных товаров по сессиям

In [271]:
buy_list = [x for x in train.buy]

Список просмотренных товаров по сессиям, в которых были покупки

In [272]:
view_list = [x for x in train.view]

Преобразование к списку кортежей (ид_товара, кол-во просмотров) по сессиям

In [273]:
for i in range(len(view_list)):
    for j in range(len(view_list[i])):
        view_list[i][j] = (view_list[i][j], freq_view_all[view_list[i][j]])

Сортировка кортежей в каждом списке (получение рекомендаций)

In [275]:
recom_list = []
for session_views in view_list:
    recom_list.append(sorted(session_views, key=lambda x: x[1], reverse=True))

In [291]:
for i, session_recoms in enumerate(recom_list):
    recom_list[i] = map(lambda x:x[0], session_recoms)

Функция оценки качества рекомендации

In [292]:
import numpy as np

In [293]:
def recom_quality(recom_list, buy_list):
    AverageRecall1 = np.mean(map(recall1, recom_list, buy_list))
    AveragePrecision1 = np.mean(map(precision1, recom_list, buy_list))
    AverageRecall5 = np.mean(map(recall5, recom_list, buy_list))
    AveragePrecision5 = np.mean(map(precision5, recom_list, buy_list))
    return [AverageRecall1, AveragePrecision1, AverageRecall5, AveragePrecision5]

In [310]:
def unique(some_list):
    unique_list = []
    for elem in some_list:
        if elem not in unique_list:
            unique_list.append(elem)
    return unique_list

In [294]:
def recall1(recom_list, buy_list):
    recom_list = set(recom_list[:1])
    buy_list = set(buy_list)
    return len(recom_list & buy_list)/float(len(buy_list))

In [295]:
def precision1(recom_list, buy_list):
    recom_list = set(recom_list[:1])
    buy_list = set(buy_list)
    return len(recom_list & buy_list)/1.0

In [312]:
def recall5(recom_list, buy_list):
    recom_list = set(unique(recom_list)[:5])
    buy_list = set(buy_list)
    return len(recom_list & buy_list)/float(len(buy_list))

In [313]:
def precision5(recom_list, buy_list):
    recom_list = set(unique(recom_list)[:5])
    buy_list = set(buy_list)
    return len(recom_list & buy_list)/5.0

Оценка качества рекомендации по популярности на тренировочной выборке

In [314]:
answer1 = recom_quality(recom_list, buy_list)

In [315]:
print ' '.join([str(x) for x in answer1])

0.442634316595 0.512195121951 0.824691824713 0.212527716186


In [381]:
def write_answer_1(answer):
    with open("recom_answer_1.txt", "w") as fout:
        fout.write(str(answer))

In [317]:
write_answer_1(' '.join([str(round(x, 2)) for x in answer1]))

### Тестовая выборка

In [342]:
test = pd.read_csv("coursera_sessions_test.txt", sep=';', header = 0, names = ["view", "buy"])

In [343]:
test.head()

,view,buy
0,"13,14,15",NaN
1,"22,23",NaN
2,"28,29,30,31,32,33",NaN
3,"40,41",NaN
4,"43,44,43,45,43,45,43,46",NaN


In [344]:
test.dropna(inplace = True)

In [345]:
test.head()

,view,buy
6,"63,68,69,70,66,61,59,61,66,68","66,63"
13,"158,159,160,159,161,162",162
18,"200,201,202,203,204","201,205"
33,"371,372,371","371,373"
39,422,422


In [346]:
test.view = map(to_list, test.view)
test.buy = map(to_list, test.buy)

In [347]:
test.head()

,view,buy
6,"[63, 68, 69, 70, 66, 61, 59, 61, 66, 68]","[66, 63]"
13,"[158, 159, 160, 159, 161, 162]",[162]
18,"[200, 201, 202, 203, 204]","[201, 205]"
33,"[371, 372, 371]","[371, 373]"
39,[422],[422]


In [348]:
test_view_list = [x for x in test.view]
test_buy_list = [x for x in test.buy]

Преобразование к списку кортежей (ид_товара, кол-во просмотров) по сессиям

In [349]:
for i in range(len(test_view_list)):
    for j in range(len(test_view_list[i])):
        try:
            test_view_list[i][j] = (test_view_list[i][j], freq_view_all[test_view_list[i][j]])
        except:
            test_view_list[i][j] = (test_view_list[i][j], 0)

In [351]:
test.head()

,view,buy
6,"[(63, 6), (68, 2), (69, 0), (70, 0), (66, 2), ...","[66, 63]"
13,"[(158, 641), (159, 81), (160, 92), (159, 81), ...",[162]
18,"[(200, 18), (201, 12), (202, 66), (203, 19), (...","[201, 205]"
33,"[(371, 9), (372, 5), (371, 9)]","[371, 373]"
39,"[(422, 60)]",[422]


In [357]:
test_recom_list = []
for session_views in test_view_list:
    test_recom_list.append(sorted(session_views, key=lambda x: x[1], reverse=True))

In [358]:
for i, session_recoms in enumerate(test_recom_list):
    test_recom_list[i] = map(lambda x:x[0], session_recoms)

Оценка качества рекомендации по популярности на тестовой выборке

In [360]:
answer2 = recom_quality(test_recom_list, test_buy_list)

In [361]:
print ' '.join([str(x) for x in answer2])

0.417332662033 0.481309686221 0.800034066354 0.203765347885


In [362]:
def write_answer_2(answer):
    with open("recom_answer_2.txt", "w") as fout:
        fout.write(str(answer))

In [363]:
write_answer_2(' '.join([str(round(x, 2)) for x in answer2]))

## Сортировка по покупаемости

### Тренировочная выборка

In [364]:
train = pd.read_csv("coursera_sessions_train.txt", sep=';', header = 0, names = ["view", "buy"])

In [365]:
train.head()

,view,buy
0,"9,10,11,9,11,12,9,11",NaN
1,"16,17,18,19,20,21",NaN
2,"24,25,26,27,24",NaN
3,"34,35,36,34,37,35,36,37,38,39,38,39",NaN
4,42,NaN


In [366]:
train.dropna(inplace = True)

In [367]:
train.head()

,view,buy
6,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63"
9,"84,85,86,87,88,89,84,90,91,92,93,86",86
18,"138,198,199,127",199
29,"303,304,305,306,307,308,309,310,311,312",303
32,"352,353,352",352


In [368]:
train.view = map(to_list, train.view)
train.buy = map(to_list, train.buy)

In [371]:
train.head()

,view,buy
6,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]"
9,"[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]",[86]
18,"[138, 198, 199, 127]",[199]
29,"[303, 304, 305, 306, 307, 308, 309, 310, 311, ...",[303]
32,"[352, 353, 352]",[352]


Подсчет частот покупок товаров

In [369]:
freq_buy = dict()

for session_buys in train.buy:
    for i in range(len(session_buys)):
        if session_buys[i] not in freq_buy:
            freq_buy[session_buys[i]] = 1
        else:
            freq_buy[session_buys[i]] += 1

In [373]:
buy_list = [x for x in train.buy]
view_list = [x for x in train.view]

for i in range(len(view_list)):
    for j in range(len(view_list[i])):
        try:
            view_list[i][j] = (view_list[i][j], freq_buy[view_list[i][j]])
        except:
            view_list[i][j] = (view_list[i][j], 0)

In [375]:
train.head()

,view,buy
6,"[(59, 0), (60, 1), (61, 0), (62, 0), (60, 1), ...","[67, 60, 63]"
9,"[(84, 0), (85, 1), (86, 2), (87, 0), (88, 0), ...",[86]
18,"[(138, 1), (198, 0), (199, 1), (127, 1)]",[199]
29,"[(303, 1), (304, 0), (305, 0), (306, 0), (307,...",[303]
32,"[(352, 2), (353, 0), (352, 2)]",[352]


Сортировка кортежей в каждом списке (получение рекомендаций)

In [376]:
recom_list = []
for session_views in view_list:
    recom_list.append(sorted(session_views, key=lambda x: x[1], reverse=True))

In [377]:
for i, session_recoms in enumerate(recom_list):
    recom_list[i] = map(lambda x:x[0], session_recoms)

Оценка качества рекомендации по покупаемости на тренировочной выборке

In [378]:
answer3 = recom_quality(recom_list, buy_list)

In [382]:
print ' '.join([str(x) for x in answer3])

0.688449492427 0.80376940133 0.926307302423 0.252549889135


In [383]:
def write_answer_3(answer):
    with open("recom_answer_3.txt", "w") as fout:
        fout.write(str(answer))

In [384]:
write_answer_3(' '.join([str(round(x, 2)) for x in answer3]))

### Тестовая выборка

In [387]:
test = pd.read_csv("coursera_sessions_test.txt", sep=';', header = 0, names = ["view", "buy"])

test.dropna(inplace = True)

test.view = map(to_list, test.view)
test.buy = map(to_list, test.buy)

In [388]:
test_view_list = [x for x in test.view]
test_buy_list = [x for x in test.buy]

for i in range(len(test_view_list)):
    for j in range(len(test_view_list[i])):
        try:
            test_view_list[i][j] = (test_view_list[i][j], freq_buy[test_view_list[i][j]])
        except:
            test_view_list[i][j] = (test_view_list[i][j], 0)

In [389]:
test_recom_list = []
for session_views in test_view_list:
    test_recom_list.append(sorted(session_views, key=lambda x: x[1], reverse=True))

In [390]:
for i, session_recoms in enumerate(test_recom_list):
    test_recom_list[i] = map(lambda x:x[0], session_recoms)

Оценка качества рекомендации по покупаемости на тестовой выборке

In [391]:
answer4 = recom_quality(test_recom_list, test_buy_list)

In [392]:
print ' '.join([str(x) for x in answer4])

0.460620166666 0.527694406548 0.820187433749 0.210095497954


In [393]:
def write_answer_4(answer):
    with open("recom_answer_4.txt", "w") as fout:
        fout.write(str(answer))

In [394]:
write_answer_4(' '.join([str(round(x, 2)) for x in answer4]))